<a href="https://colab.research.google.com/github/sumitprakash0601/aws_code_deploy/blob/master/tutorial_writer_v0_1_4_openAI_v0_4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Create A Crew of Agents To Write Tutorial On A Specified Topic



In [1]:
!pip install numpy==1.25.2
!pip install crewai==0.28.8 crewai_tools==0.1.6 langchain_community==0.0.29

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.8/10.8 MB 59.4 MB/s eta 0:00:00
  Installing build dependencies ... done
  error: subprocess-exited-with-error
  
  × Getting requirements to build wheel did not run successfully.
  │ exit code: 1
  ╰─> See above for output.
  
  note: This error originates from a subprocess, and is likely not a problem with pip.
  Getting requirements to build wheel ... error
error: subprocess-exited-with-error

× Getting requirements to build wheel did not run successfully.
│ exit code: 1
╰─> See above for output.

note: This error originates from a subprocess, and is likely not a problem with pip.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.9/40.9 kB 3.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.0/61.0 kB 5.7 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 7.6 MB

In [1]:
# Warning control
import warnings
warnings.filterwarnings('ignore')

In [2]:
from crewai import Agent, Task, Crew

In [3]:
import os
from google.colab import userdata
openaikey = userdata.get('OpenAPIKey')
os.environ['OPENAI_API_KEY'] = openaikey
os.environ["OPENAI_MODEL_NAME"] = 'gpt-3.5-turbo'

# Agent: Planner

In [4]:
tutorial_outline_creator = Agent(
    role="Tutorial Outline Creator",
    goal=(
        "To create a clear, concise, and logically structured outline for a tutorial on the subject of "
        "{tutorial_subject}. The outline should simplify the subject and make it accessible to beginners."
    ),
    backstory=(
        "You are an expert at instructional design. Your task is to create an outline for a tutorial on the "
        "topic: {tutorial_subject}. \n\n"

        "This outline will be passed to the Tutorial Writer agent, who will use your structure to write the "
        "complete tutorial. Make sure your outline is clear, logical, and avoids jargon."
    ),
    allow_delegation=False,
    verbose=True
)

### Agent: Writer

In [5]:
tutorial_writer = Agent(
    role="Tutorial Writer",
    goal=(
        "To write a clear, engaging, and beginner-friendly tutorial on the subject: {tutorial_subject}, "
        "based on the outline provided. You add detail to the tutorial based on the outline created by the outline creator"
    ),
    backstory=(
        "You are an experienced educator and writer. Your task is to write a tutorial on the topic: "
        "{tutorial_subject}. You will follow the detailed outline provided by the Tutorial Outline Creator.\n\n"

        "Your goal is to make complex ideas approachable. Avoid jargon unless it's necessary, and if used, explain it simply."
    ),
    allow_delegation=False,
    verbose=True
)

### Agent: Editor

In [6]:
tutorial_editor = Agent(
    role="Tutorial Editor",
    goal=(
        "To edit and refine the tutorial written by the Tutorial Writer to ensure it is "
        "clear, grammatically correct, logically structured, and easy for beginners to understand."
    ),
    backstory=(
        "You are an experienced editor specializing in educational content. You have received a tutorial "
        "written by the Tutorial Writer based on a structured outline. Your task is to carefully review the "
        "tutorial and make necessary edits to ensure high quality.\n\n"

        "If something is unclear or seems missing, suggest improvements or reword it for clarity. "
        "Your edits should preserve the author's intent while improving readability and impact."
    ),
    allow_delegation=False,
    verbose=True
)

# Task: tutorial_outline_creation

In [14]:
tutorial_outline_creation = Task(
    description=(
        "Your task is to create a detailed outline for a tutorial on the subject: {tutorial_subject}.\n\n"
        "Please follow these guidelines:\n"
        "1. The tutorial is intended for beginners. Avoid technical jargon and use simple, accessible language.\n"
        "2. The outline should include:\n"
        "   - A brief introduction to the subject\n"
        "   - The *top 5 key topics* (no more)\n"
        "   - A section explaining the relevance and real-world applications of the subject.\n"
        "   - A short conclusion or summary\n"
        "3. For each of the 5 key topics, provide:\n"
        "   - A clear title\n"
        "   - Detail of what should be covered under this topic\n"
        "   - Key learning objectives or questions to answer\n"
        "4. Include a section explaining the relevance and real-world applications of the subject."
        "5. Include a section explaining the cost impact of the subject."
    ),
    expected_output=(
        "A structured tutorial outline that includes:\n"
        "- An introduction to {tutorial_subject}\n"
        "- The top 5 key topics with titles, content descriptions, and learning objectives\n"
        "- A short conclusion summarizing the key takeaways\n"
        "- A section on the relevance and real-world applications and cost of the subject"
    ),
    agent=tutorial_outline_creator,
)

# Task: Tutorial Writing

In [15]:
tutorial_writing = Task(
    description=(
        "Use the detailed tutorial outline (created by the Tutorial Outline Creator) to write a complete tutorial "
        "on the subject: {tutorial_subject}.\n\n"
        "Please follow these guidelines:\n"
        "1. The tutorial must follow the structure and key points provided in the outline.\n"
        "2. Use simple, beginner-friendly language. Avoid technical jargon.\n"
        "3. Explain each topic clearly, including examples or analogies where relevant.\n"
        "4. Use figures or diagrams where appropriate (describe them in words).\n"
        "5. Keep the tutorial detailed\n"
        "6. Ensure the tutorial is grammatically correct and free from spelling errors.\n"
        "7. Format the tutorial using markdown (with clear headings, bullet points, etc.)."
    ),
    expected_output=(
        "A complete, well-written tutorial in markdown format that:\n"
        "- Is structured by introduction, key topics, and conclusion\n"
        "- Each Key Topic explained in detail"
        "- Uses simple language and examples\n"
        "- Suggests diagrams where needed\n"
        "- Is clean, clear, and ready for publishing"
    ),
    agent=tutorial_writer,
)

# Task: Tutorial Editing

In [16]:
tutorial_editing = Task(
    description=(
        "You are assigned to edit the tutorial written by the Tutorial Writer on the subject: {tutorial_subject}.\n\n"
        "Please perform a thorough review and apply the following checks:\n"
        "1. Correct any grammatical and spelling errors.\n"
        "2. Ensure that the language is simple, clear, and accessible to beginners.\n"
        "3. Maintain consistency in tone, voice, and formatting throughout the tutorial.\n"
        "4. Ensure that examples and figure descriptions (if any) are understandable and effectively placed.\n"
        "5. Preserve the original structure and markdown formatting.\n"
        "6. Make small improvements to clarity or flow where necessary — rewrite awkward or complex sentences.\n"
        "7. Do not add new content or topics not present in the original.\n"
    ),
    expected_output=(
        "An edited version of the tutorial in markdown format, ready for publication.\n"
        "The tutorial should:\n"
        "- Be free from grammatical and spelling errors\n"
        "- Maintain clear, simple language for beginners\n"
        "- Retain formatting and structure from the original version\n"
        "- Be publication-ready with a polished, professional tone"
    ),
    agent=tutorial_editor
)


# Crew

In [17]:
crew = Crew(
    agents=[tutorial_outline_creator, tutorial_writer, tutorial_editor],
    tasks=[tutorial_outline_creation, tutorial_writing,tutorial_editing],
    verbose=2
)

# Let us now kick off the crew in motion

In [21]:
tutorial = crew.kickoff(inputs={"tutorial_subject": "Fargate Technology"})

 [DEBUG]: == Working Agent: Tutorial Outline Creator
 [INFO]: == Starting Task: Your task is to create a detailed outline for a tutorial on the subject: Fargate Technology.

Please follow these guidelines:
1. The tutorial is intended for beginners. Avoid technical jargon and use simple, accessible language.
2. The outline should include:
   - A brief introduction to the subject
   - The *top 5 key topics* (no more)
   - A section explaining the relevance and real-world applications of the subject.
   - A short conclusion or summary
3. For each of the 5 key topics, provide:
   - A clear title
   - Detail of what should be covered under this topic
   - Key learning objectives or questions to answer
4. Include a section explaining the relevance and real-world applications of the subject.5. Include a section explaining the cost impact of the subject.


> Entering new CrewAgentExecutor chain...
I now can give a great answer

Final Answer: 

Tutorial Outline: Fargate Technology

Introduction

In [20]:
from IPython.display import Markdown
Markdown(tutorial)

```markdown
# Fargate Technology Tutorial

## Introduction

Fargate Technology is a groundbreaking service provided by Amazon Web Services (AWS) that simplifies the process of deploying containerized applications. By abstracting the underlying infrastructure, Fargate enables developers to concentrate on building and running applications without the hassle of managing servers.

## Key Topic 1: What is Fargate Technology?

- **Definition**: Fargate Technology is a serverless compute engine for containers that allows you to run Docker containers without needing to handle the underlying infrastructure.
- **Difference from traditional EC2 instances**: Unlike traditional EC2 instances that require provisioning and server management, Fargate abstracts this layer, allowing a focus solely on applications.
- **Learning Objectives**: Understanding Fargate Technology helps in grasping container management basics and simplifying the deployment process.

## Key Topic 2: Benefits of Fargate Technology

- **Cost Efficiency and Scalability**: Fargate Technology offers cost efficiency by charging based on the resources used and provides seamless scalability as application demands increase.
- **Increased Security and Reduced Operational Overhead**: Fargate ensures a secure container environment and reduces operational overhead by managing server tasks.
- **Learning Objectives**: Recognize the advantages of using Fargate Technology for deploying applications and its benefits for organizations.

## Key Topic 3: How to Deploy Applications with Fargate

- **Step-by-Step Guide**: Deploying applications with Fargate involves creating task definitions specifying container image, CPU, memory requirements, and networking configuration. Container definitions are also defined within task definitions to specify container runtime.
- **Learning Objectives**: Following a step-by-step guide on deploying applications with Fargate helps in effectively utilizing this technology.

## Key Topic 4: Monitoring and Scaling with Fargate

- **Overview of Monitoring Tools**: Fargate provides built-in monitoring tools for tracking container and application performance.
- **Auto-Scaling Capabilities**: Fargate offers auto-scaling for adjusting the number of running containers based on workload.
- **Best Practices**: Implementing best practices for monitoring and scaling containerized applications ensures efficient performance and resource utilization.
- **Learning Objectives**: Understand how to monitor and scale applications effectively with Fargate.

## Key Topic 5: Integration with Other AWS Services

- **Integration with ECS and EKS**: Fargate seamlessly integrates with AWS services like Elastic Container Service (ECS) and Elastic Kubernetes Service (EKS) for comprehensive container management.
- **Benefits of Integration**: Combining Fargate with other AWS services provides additional functionalities and streamlines application deployment.
- **Learning Objectives**: Explore integrating Fargate with other AWS services to enhance application functionality.

## Relevance and Real-World Applications

- **Real-World Examples**: Industries like e-commerce, healthcare, and finance use Fargate Technology to streamline deployment processes and improve operational efficiency.
- **Streamlining Deployment**: Fargate simplifies deployment by abstracting away infrastructure management tasks, allowing businesses to focus on core applications.

## Cost Impact

- **Cost Structure**: Fargate follows a pay-as-you-go pricing model, where you pay based on compute resources consumed by containers.
- **Cost Savings**: Comparing cost savings with Fargate versus traditional methods, businesses can optimize cloud spending and achieve better resource utilization.
- **Optimizing Cloud Spending**: Fargate helps businesses optimize cloud spending by offering cost-effective and scalable container management solutions.

## Conclusion

In conclusion, Fargate Technology is a game-changer in containerized applications, offering simplicity, scalability, security, and cost efficiency. By exploring the key topics in this tutorial, you can leverage the power of Fargate for effective application deployment and management. Encourage further exploration of Fargate Technology in various industries to unlock its full potential.
```